In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

## Lets get the data, model and setup trainnig code

In [3]:
train_loader = DataLoader(datasets.CIFAR10("./", train=True, transform=transforms.ToTensor(), download=True), batch_size=128, shuffle=True)
test_loader = DataLoader(datasets.CIFAR10("./", train=False, transform=transforms.ToTensor(), download=True), batch_size=128, shuffle=False)

print(f"Training images {len(train_loader.dataset)}, Test images {len(test_loader.dataset)}")

Files already downloaded and verified
Files already downloaded and verified
Training images 50000, Test images 10000


In [4]:

from einops import rearrange, repeat
from einops.layers.torch import Rearrange

# helpers

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

# classes

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
            nn.Linear(patch_dim, dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)


In [6]:
model = ViT(image_size = 256,
    patch_size = 32,
    num_classes = 1000,
    dim = 1024,
    depth = 6,
    heads = 8,
    mlp_dim = 2048
).cuda()
print(model)

epochs = 15
lr = 0.1

optimizer = optim.SGD(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
lrs = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

ViT(
  (to_patch_embedding): Sequential(
    (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=32, p2=32)
    (1): Linear(in_features=3072, out_features=1024, bias=True)
  )
  (dropout): Dropout(p=0.0, inplace=False)
  (transformer): Transformer(
    (layers): ModuleList(
      (0): ModuleList(
        (0): PreNorm(
          (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fn): Attention(
            (attend): Softmax(dim=-1)
            (to_qkv): Linear(in_features=1024, out_features=1536, bias=False)
            (to_out): Sequential(
              (0): Linear(in_features=512, out_features=1024, bias=True)
              (1): Dropout(p=0.0, inplace=False)
            )
          )
        )
        (1): PreNorm(
          (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fn): FeedForward(
            (net): Sequential(
              (0): Linear(in_features=1024, out_features=2048, bias=True)
              (1): GELU()
         



```
# This is formatted as code
```

## Training

In [6]:
def get_acc(model, loader):
  correct = 0
  total = 0
  for img, label in loader:
    correct += torch.sum(torch.argmax(model(img.cuda()), -1).cpu() == label).item()
    total += len(img)
  return 100*correct/total

In [7]:
for e in range(epochs):
  print("lr", optimizer.param_groups[0]["lr"])
  for img, label in train_loader:
    # print(img.shape, label.shape)
    out = model(img.cuda())
    # print(out.shape)
    optimizer.zero_grad()
    loss = criterion(out, label.cuda())
    loss.backward()
    optimizer.step()
  lrs.step()
  print(f"Epoch {e}, training accuracy {get_acc(model, train_loader)}, test accuracy {get_acc(model, test_loader)}")

lr 0.1
Epoch 0, training accuracy 97.05666666666667, test accuracy 97.15
lr 0.09890738003669029
Epoch 1, training accuracy 98.35333333333334, test accuracy 98.37
lr 0.09567727288213004
Epoch 2, training accuracy 98.69, test accuracy 98.59
lr 0.09045084971874738
Epoch 3, training accuracy 98.995, test accuracy 98.95
lr 0.08345653031794292
Epoch 4, training accuracy 99.21833333333333, test accuracy 98.98
lr 0.07500000000000001
Epoch 5, training accuracy 99.33666666666667, test accuracy 99.02
lr 0.06545084971874739
Epoch 6, training accuracy 99.345, test accuracy 98.98
lr 0.05522642316338269
Epoch 7, training accuracy 99.53666666666666, test accuracy 99.04
lr 0.04477357683661735
Epoch 8, training accuracy 99.66833333333334, test accuracy 99.16
lr 0.03454915028125265
Epoch 9, training accuracy 99.695, test accuracy 99.18
lr 0.02500000000000002
Epoch 10, training accuracy 99.73833333333333, test accuracy 99.16
lr 0.01654346968205711
Epoch 11, training accuracy 99.76, test accuracy 99.15
lr 

## Extract weights

In [8]:
params = [(name, p.data.cpu().numpy()) for (name, p) in model.named_parameters()]

In [9]:
for (name, p) in params:
  print(f"Layer {name.split('.')[0]}, type {name.split('.')[1]}, shape {p.shape}")

Layer layer1, type weight, shape (20, 1, 5, 5)
Layer layer1, type bias, shape (20,)
Layer layer2, type weight, shape (50, 20, 5, 5)
Layer layer2, type bias, shape (50,)
Layer layer3, type weight, shape (500, 800)
Layer layer3, type bias, shape (500,)
Layer layer4, type weight, shape (10, 500)
Layer layer4, type bias, shape (10,)


In [10]:
#print(params)

## Visualize hidden activations

In [11]:
# print(model.children())
# out = list(model.children())[0](img.cuda()).data.cpu().numpy()

In [12]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# for _ in range(out.shape[1]):
#   plt.figure(figsize=(1, 1))
#   plt.imshow(out[0, 0], cmap="gray")

In [13]:
for img, label in train_loader:
  break

In [14]:
print(label)

tensor([8, 7, 1, 1, 7, 9, 1, 7, 7, 4, 7, 7, 7, 7, 2, 0, 3, 4, 4, 4, 1, 4, 2, 6,
        2, 3, 3, 7, 5, 7, 5, 0, 8, 3, 3, 5, 4, 7, 3, 4, 5, 9, 8, 1, 9, 1, 7, 9,
        3, 6, 3, 6, 7, 4, 2, 1, 5, 1, 1, 8, 9, 7, 5, 5, 4, 5, 6, 3, 9, 1, 8, 2,
        3, 8, 5, 9, 1, 8, 3, 8, 7, 8, 6, 1, 3, 4, 6, 6, 0, 2, 3, 3, 9, 7, 5, 9,
        1, 4, 6, 6, 0, 4, 4, 8, 6, 6, 8, 6, 2, 6, 3, 5, 6, 5, 7, 0, 5, 0, 5, 2,
        0, 9, 2, 2, 1, 5, 4, 8])


In [15]:

path = "./LeNet/"
import os
if not os.path.isdir(path):
    os.mkdir(path)
np.savetxt(fname=path+"label", delimiter=" ", X=label.tolist())
print(get_acc(model, ([img,label],)))

100.0


In [16]:

np.savetxt(fname=path+"input_0", delimiter=" ", X=img.cuda().view(-1, 784).tolist())
np.savetxt(fname=path+"outputlayer1_0", delimiter=" ", X=model.output(img.cuda().view(128, 1, 28, 28))[0].data.cpu().view(-1))
np.savetxt(fname=path+"outputlayer2_0", delimiter=" ", X=model.output(img.cuda().view(128, 1, 28, 28))[1].tolist())
np.savetxt(fname=path+"outputlayer3_0", delimiter=" ", X=model.output(img.cuda().view(128, 1, 28, 28))[2].tolist())
np.savetxt(fname=path+"outputlayer4_0", delimiter=" ", X=model.output(img.cuda().view(128, 1, 28, 28))[3].tolist())

np.savetxt(fname=path+"weight1_0", delimiter=" ", X=params[0][1].reshape(5*5*1, 20).tolist())
np.savetxt(fname=path+"bias1_0", delimiter=" ", X=params[1][1].tolist())
np.savetxt(fname=path+"weight2_0", delimiter=" ", X=params[2][1].reshape(5*5*20, 50).tolist())
np.savetxt(fname=path+"bias2_0", delimiter=" ", X=params[3][1].tolist())
np.savetxt(fname=path+"weight3_0", delimiter=" ", X=params[4][1].tolist())
np.savetxt(fname=path+"bias3_0", delimiter=" ", X=params[5][1].tolist())
np.savetxt(fname=path+"weight4_0", delimiter=" ", X=params[6][1].tolist())
np.savetxt(fname=path+"bias4_0", delimiter=" ", X=params[7][1].tolist())
